In [1]:
!pip install evaluate transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import argparse
import csv

import evaluate
import numpy as np
import torch
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer


#toxicity = evaluate.load("ybelkada/toxicity", "DaNLP/da-electra-hatespeech-detection", module_type="measurement")
toxicity = evaluate.load("toxicity", 'Overfit-GM/mdeberta-v3-base-offensive', module_type="measurement",)
ds = load_dataset("Overfit-GM/turkish-toxic-language", split="train")

#parser = argparse.ArgumentParser(description="Evaluate de-toxified models")
#parser.add_argument("--model_type", default="all", type=str, help="Relative path to the source model folder")
#parser.add_argument("--output_file", default="toxicity.csv", type=str, help="Relative path to the source model folder")
#parser.add_argument("--batch_size", default=64, type=int, help="Batch size")
#parser.add_argument("--num_samples", default=400, type=int, help="Number of samples")
#parser.add_argument("--context_length", default=2000, type=int, help="Number of samples")
#parser.add_argument("--max_new_tokens", default=30, type=int, help="Max new tokens for generation")
#args = parser.parse_args()



MODELS_TO_TEST = ["redrussianarmy/gpt2-turkish-cased","yankihue/gpt2-tr-detoxified-final"]


NUM_SAMPLES = 400
BATCH_SIZE = 64
output_file = "toxicity_comparison.csv"
max_new_tokens = 30
context_length = 2000
device = torch.cuda.current_device() if torch.cuda.is_available() else "cpu"

# consider only toxic prompts
ds = ds.filter(lambda x: x["is_toxic"] == 1)

toxicities = {}

# open a csv file
file = open(f"{output_file}", "w", newline="")
writer = csv.writer(file)
# add first rows
writer.writerow(["model_id", "mean_toxicity", "std_toxicity"])


for model_id in tqdm(MODELS_TO_TEST):
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"": device}, torch_dtype=torch.bfloat16)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.eos_token_id = model.config.eos_token_id
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"
    input_texts = []

    for i, example in enumerate(ds):
        # set seed
        torch.manual_seed(42)

        input_text = example["text"]
        input_texts.append(input_text[:2000])

        if i > NUM_SAMPLES:
            break

        if (i + 1) % BATCH_SIZE == 0:
            inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
            inputs.input_ids = inputs.input_ids[:context_length]
            inputs.attention_mask = inputs.attention_mask[:context_length]
            outputs = model.generate(**inputs, do_sample=True, max_new_tokens=max_new_tokens, use_cache=True)
            generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            generated_texts = [
                generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)
            ]
            toxicity_score = toxicity.compute(predictions=generated_texts,toxic_label="INSULT")
            input_texts = []

            if model_id not in toxicities:
                toxicities[model_id] = []
            toxicities[model_id].extend(toxicity_score["toxicity"])

    # last batch
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True).to(device)
    outputs = model.generate(**inputs, do_sample=True, max_new_tokens=30)
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    generated_texts = [generated_text.replace(input_texts[i], "") for i, generated_text in enumerate(generated_texts)]
    toxicity_score = toxicity.compute(predictions=generated_texts,toxic_label="INSULT")
    toxicities[model_id].extend(toxicity_score["toxicity"])

    # compute mean & std using np
    mean = np.mean(toxicities[model_id])
    std = np.std(toxicities[model_id])

    # save to file
    writer.writerow([model_id, mean, std])

    # print
    print(f"Model: {model_id} - Mean: {mean} - Std: {std}")

    model = None
    torch.cuda.empty_cache()

# close file
file.close()


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
  0%|          | 0/2 [00:00<?, ?it/s]Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
 50%|█████     | 1/2 [43:3

Model: redrussianarmy/gpt2-turkish-cased - Mean: 0.00042136561389035065 - Std: 0.005402000393132089


Some weights of the model checkpoint at yankihue/gpt2-tr-detoxified-final were not used when initializing GPT2LMHeadModel: ['v_head.summary.weight', 'v_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Set

Model: yankihue/gpt2-tr-detoxified-final - Mean: 9.80034803831414e-06 - Std: 3.655606379583462e-06


In [3]:
#from transformers import AutoModelForSequenceClassification

#model2 = AutoModelForSequenceClassification.from_pretrained("Overfit-GM/mdeberta-v3-base-offensive")

In [4]:
#model2.config.id2label

In [5]:
#toxicity = evaluate.load("toxicity", 'Overfit-GM/mdeberta-v3-base-offensive', module_type="measurement",)

#toxicity.compute(predictions=["allah senin belanı versin iğrenç herif", "NEDEN LAN NEDEN OÇLUK YAPIYORSUN PİÇ"],toxic_label="INSULT")